In [1]:
import boto3
import botocore
import pickle

BUCKET_NAME = 'ts-dev-cs-training-data'
PREFIX = 'rbp-research/data/e2e/2017-09-11/'

s3 = boto3.resource('s3').Bucket(BUCKET_NAME)

In [4]:
# ssh -NL 5444:tradeshift.slave.db.eu-west-1.prod.ts.sv:5432 fli@sqltunnel.eu-west-1.prod.ts.sv
%load_ext sql
%sql postgresql+psycopg2://fli@localhost:5444/tradeshift?sslmode=require
%config SqlMagic.feedback=False

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [5]:
def get_a_train_file_key(s3, prefix):
    for o in s3.objects.filter(Prefix=prefix, Delimiter='/'):
        yield o.key

In [ ]:
gen = get_a_train_file_key(s3, PREFIX)


doc_sender_dict = {}

i = 0
for s in gen:
    i = i + 1
    key = s.replace('.','/').split('/')[-3]
    try:
        sender_id = %sql select groupid from documents where uuid = :key
        sender_id = str(sender_id.dict()['groupid'][0])
    except:
        sender_id = 'Unknown'
    
    doc_sender_dict[key] = sender_id
    
    if i % 10000 == 1:
        print('Doc number {}. doc_sender_dict {}'.format(i, len(doc_sender_dict)))
    
    #if i == 68248:
        #break

# pickle.dump(doc_sender_dict, open('doc_sender_dict.pickle', 'wb'))
import json
with open('doc_sender_dict.json', 'w') as f:
        json.dump(doc_sender_dict, f)

Doc number 1. doc_sender_dict 1


In [6]:
import pickle
pickle.dump(doc_sender_dict, open('doc_sender_dict.pickle', 'wb'))

In [7]:
len(doc_sender_dict)

68248

In [26]:
def get_sender_name_via_id(doc_id):
    import boto3
    import botocore
    import json
    
    b_name = 'ts-dev-cs-training-data'
    prefix = 'cache_v2_individual/ubl1/hocr13/ubllight1/hocrlight1/sendertrainingmatrices20170819/data/'
    key = prefix + doc_id + '.training'
    
    obj = boto3.resource('s3').Object(b_name, key)
    doc_str = obj.get()['Body'].read().decode('utf-8')
    doc_json = json.loads(doc_str)
    
    return doc_json['sender']

In [27]:
get_sender_name_via_id('00001593-256e-46f5-b0a1-bd0c09981c3a')

'UTOPIA TABLEWARE LTD'